In [ ]:
!pip install pandas
!pip install matplotlib
!pip install requests
!pip install beautifulsoup4
!pip install urllib3
!pip install seaborn
!pip install wordcloud


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
import requests 
from bs4 import BeautifulSoup
import urllib.request
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline


def display_menu():
  print("1. Top 10 most valuable players")
  print("2. How does BMI affect the overall score?")
  print("3. Which country has the maximum number of players?")
  print("4. What players have the highest overall?")
  print("5. Which league has the most valuable players?")
  print("6. Highest rated club team")
  print("7. What are the top 20 nations that the players represent?")
  print("8. What is the correlation between a player's potential and a player's age?")
  print("9. What are the different positions and the number of players playing them?")
  print("10. Correlation between age and wage")
  print("11. Is there a significant pattern among the 15 most valuable players' age and wage?")
  print("12. Correlation between weakfoot and passing, shooting, dribbling")
  print("13. Exit")


def print_correlation_message(correlation): # designed by Jemil for common use
  if correlation == 1.0:
    print("Perfect possitive association")
  elif correlation >= 0.8 and correlation < 1.0:
    print("Very strong positive association")
  elif correlation >= 0.6 and correlation < 0.8:
    print("Strong positive association")
  elif correlation >= 0.4 and correlation < 0.6:
    print("Moderate positive association")
  elif correlation >= 0.2 and correlation < 0.4:
    print("Weak positive association")
  elif correlation > 0.0 and correlation < 0.2:
    print("Very weak positive association")
  elif correlation == 0.0:
    print("No positive or negative association")
  elif correlation > -0.2 and correlation < 0.0:
    print("Very weak negative association")
  elif correlation > -0.4 and correlation <= -0.2:
    print("Weak negative association")
  elif correlation > -0.6 and correlation <= -0.4:
    print("Moderate negative association")
  elif correlation > -0.8 and correlation <= -0.6:
    print("Strong negative association")
  elif correlation > -1.0 and correlation <= -0.8:
    print("Very strong negative association")
  elif correlation == -1.0:
    print("Perfect negative association")
  else:
    print("Invalid correlation value")


def choice_1(): # Jemil Question 1

  df = pd.read_csv("players_22.csv", low_memory=False)

  player_value_df = df.sort_values(by="value_eur", ascending=False).head(10)

  player_names = player_value_df["long_name"].tolist()
  player_values = player_value_df["value_eur"].tolist()

  index = 0

  while index < len(player_names):
    formatted_value = "{:,}".format(player_values[index])
    print(f"{index + 1}. {player_names[index]}: €{formatted_value}")
    index += 1  

  player_value_df.to_json("player_details.json", orient="records", indent=4)
  print("\nplayer_details.json successfully created!")
  print("The above file contains the data of top 10 most valuable players in a json format")


def choice_2(): # Jemil Question 2
  
  df = pd.read_csv("players_22.csv", low_memory=False)
  
  player_height = df["height_cm"].tolist()
  player_weight = df["weight_kg"].tolist()

  player_height = [int(x) for x in player_height]
  player_weight = [int(x) for x in player_weight]

  player_bmi = []

  index = 0

  while index < len(player_height):
    bmi = player_weight[index] / (player_height[index] / 100) ** 2
    bmi = round(bmi, 2)
    player_bmi.append(bmi)
    index += 1

  if "bmi" not in df.columns:
    df.insert(loc=13, column="bmi", value=player_bmi)

  plt.scatter(df.bmi, df.overall, s=20)
  plt.title("Scatter Plot")
  plt.xlabel("BMI")
  plt.ylabel("Overall score")
  plt.show()

  correlation = df["bmi"].corr(df["overall"])
  correlation = round(correlation, 2)

  print(f"Correlation of BMI and overall score is: {correlation}")

  print_correlation_message(correlation)


def choice_3(): # Jemil Question 3

  df = pd.read_csv("players_22.csv", low_memory=False)

  player_country = dict()

  for x in df["nationality_name"].tolist():
    if x not in player_country:
      player_country[x] = 1
    else:
      player_country[x] += 1

  player_country = dict(sorted(player_country.items(), key=lambda item: item[1], reverse=True))

  max_country = list(player_country.keys())[0]
  max_players = list(player_country.values())[0]

  print(f"{max_country}: {max_players}")

  htmldata = requests.get("https://en.wikipedia.org/wiki/" + max_country).text

  soup = BeautifulSoup(htmldata, "html.parser") 
  for item in soup.find_all("img"):
    if "Flag" in item["src"] or "flag" in item["src"]:
      max_country_flag = item["src"]
      break

  max_country_flag_link = "https:" + max_country_flag

  urllib.request.urlretrieve(max_country_flag_link, "flag.png")

  img = mpimg.imread("flag.png")
  imgplot = plt.imshow(img)
  plt.axis("off")
  plt.show()


def choice_4(): # Matthew Question 1

  df = pd.read_csv("players_22.csv", low_memory=False)

  player_overall_df = df.sort_values(by="overall", ascending=False).head(10)

  player_names = player_overall_df["long_name"].tolist()
  player_overalls = player_overall_df["overall"].tolist()

  index = 0

  while index < len(player_names):
    formatted_value = "{:,}".format(player_overalls[index])
    print(f"{index + 1}. {player_names[index]}: {formatted_value}")
    index += 1  


def choice_5(): # Matthew Question 2

  x=["French Ligue","German 1","English Premier","Spain Premier","Bundesliga","Italian Serie A"]

  y=[10,13,37,23,13,14]

  c=["red","green","blue","orange"]
  plt.bar(x,y,width=.25,color=c)

  plt.xlabel("Leagues")
  plt.ylabel("Amount of top 100 players")
  plt.title("Top Leagues")
  plt.show()
  f = plt.figure()
  f.set_figwidth(5)
  f.set_figheight(1)


def choice_6(): # Matthew Question 3

  x=["Manchester City","Liverpool FC","Chelsea","FC Bayern Munchen","Tottenham Hotspur"]

  y=[100,97,95,94,93]

  c=["red","green","blue","orange"]
  plt.bar(x,y,width=.25,color=c)

  plt.xlabel("Teams")
  plt.ylabel("Value of Team")
  plt.title("Top Rated Teams in Soccer")
  plt.show()


def choice_7(): # Manita Question 1

  df = pd.read_csv("players_22.csv", low_memory=False)
  top_nations = df.nationality_name.value_counts().head(20)
  print("The top 20 nations that the players represent: ")
  print(top_nations)

  plt.figure(figsize=(8,6))
  plt.xlabel("Nationality Name")
  plt.ylabel("Number of players")
  plt.xticks(rotation= 80)
  plt.title("Top 20 Nations", fontsize=20)
  sns.barplot(x = top_nations.index, y = top_nations.values)
  plt.show()

  print("The wordcloud displaying nationality names: ")
  nationality = df.nationality_name
  plt.subplots(figsize = (12,12))
  wordcloud = WordCloud(
      background_color = "black",
      width = 2000,
      height = 1200
      ).generate(" ".join(nationality))
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.show()


def choice_8(): # Manita Question 2

  df = pd.read_csv("players_22.csv", low_memory=False)
  players_potential = df["potential"]
  players_age = df["age"]

  fig, ax = plt.subplots(figsize=(12,8))
  ax.scatter(x = df["age"], y = df["potential"])
  plt.title("Scatterplot of Player's Age and Potential", fontsize = 20)
  plt.xlabel("Player's Age", fontsize = 14)
  plt.ylabel("Player's Potential", fontsize = 14)
  plt.show()
  
  correlation = players_age.corr(players_potential)
  print("The correlation between age and potential is: ", correlation)
  print_correlation_message(correlation)


def choice_9(): # Manita Question 3

  df = pd.read_csv("players_22.csv", low_memory=False)
  players_positions = df.player_positions.unique()
  print("The different player positions are: ")
  print(players_positions)

  numbers_of_positions = df.player_positions.nunique()
  print("The total number of combination of different player positions are: ", numbers_of_positions)

  players_count = df.player_positions.value_counts().head(20)
  print("Displaying different player position with top 20 highest number of players: ")
  print(players_count)

  quickchart_url = "https://quickchart.io/chart/create"
  post_data = {"chart": {"type" : "bar", 
                       "data": {"labels": ["CB","GK","ST","CM","LB","RB","CDM","CAM","LM","RM"],
                                "datasets": [{"label": "Number of Players in Different Positions",
                                "data": [2423, 2132, 1770, 726, 616, 576, 294, 219, 206, 196]}]}}}
  response = requests.post(quickchart_url,
                         json = post_data,
                         )
  if (response.status_code != 200):
    print("Error: ", response.text)
  else:
    chart_response = json.loads(response.text)
    print(chart_response)
  

def choice_10(): # Patricia Question 1

  df = pd.read_csv('players_22.csv', low_memory=False)
  df =df[['age','wage_eur','short_name']]
  correlation = df['age'].corr(df['wage_eur'])
  fig, ax= plt.subplots(figsize=(10,10))
  ax.scatter (x= df['age'],y= df['wage_eur'])
  plt.xlabel("Age") 
  plt.ylabel("Wage EUR")
  plt.yscale("log") 
  plt.title("Age & wage ", fontsize = 11)
  plt.show()
  print_correlation_message(correlation)


def choice_11(): # Patricia Question 2

  df = pd.read_csv('players_22.csv', low_memory=False)
  df =df[['age','wage_eur','short_name']]
  fig, ax = plt.subplots(figsize=(8,5))

  df = pd.read_csv("players_22.csv", low_memory=False)
  player_value_df = df.sort_values(by="age", ascending=False).head(10)
  
  plt.scatter(player_value_df['age'], player_value_df['wage_eur'] )
  plt.text(player_value_df.iloc[0]['age'], player_value_df.iloc[0]['wage_eur'], player_value_df.iloc[0]['short_name'])
  plt.text(player_value_df.iloc[2]['age'], player_value_df.iloc[2]['wage_eur'], player_value_df.iloc[2]['short_name'])
  plt.text(player_value_df.iloc[4]['age'], player_value_df.iloc[4]['wage_eur'], player_value_df.iloc[4]['short_name'])
  plt.text(player_value_df.iloc[5]['age'], player_value_df.iloc[5]['wage_eur'], player_value_df.iloc[5]['short_name'])
  plt.text(player_value_df.iloc[6]['age'], player_value_df.iloc[6]['wage_eur'], player_value_df.iloc[6]['short_name'])
  plt.text(player_value_df.iloc[7]['age'], player_value_df.iloc[7]['wage_eur'], player_value_df.iloc[7]['short_name'])
  plt.text(player_value_df.iloc[8]['age'], player_value_df.iloc[8]['wage_eur'], player_value_df.iloc[8]['short_name'])
  plt.text(player_value_df.iloc[9]['age'], player_value_df.iloc[9]['wage_eur'], player_value_df.iloc[9]['short_name'])

  ax.set_title("Age vs Wages of top 15")
  ax.set_ylabel("Wages in Eur")
  ax.set_xlabel('Age')

  plt.show()


def choice_12(): # Patricia Question 3
  
  df = pd.read_csv('players_22.csv', low_memory=False)
  players_passing = df["passing"]
  players_shooting = df["shooting"]
  players_dribbling = df["dribbling"]
  players_weak_foot = df["weak_foot"]

  print("Correlation between weak foot and passing: ", players_weak_foot.corr(players_passing))
  print("Correlation between weak foot and shooting: ", players_weak_foot.corr(players_shooting))
  print("Correlation between weak foot and dribbling:", players_weak_foot.corr(players_dribbling))


print("\n===================================================================================")
print("                              FIFA 22 PLAYER ANALYSIS                              ")
print("===================================================================================")
print("\nWelcome to FIFA 22 player analysis project!")
print("Each number below corresponds to an inquiry you can run about our dataset:\n")


display_menu()


while True:

  print("\n-----------------------------------------------------------------------------------")
  choice = input("Enter a number of your choice: ")
  print("-----------------------------------------------------------------------------------\n")

  if choice == "1":
    print("Top 10 most valuable players")
    choice_1()

  elif choice == "2":
    print("How does BMI affect the overall score?")
    choice_2()

  elif choice == "3":
    print("Which country has the maximum number of players?")
    choice_3()

  elif choice == "4":
    print("What players have the highest overall?")
    choice_4()
    
  elif choice == "5":
    print("Which league has the most valuable players?")
    choice_5()

  elif choice == "6":
    print("Highest rated club team")
    choice_6()

  elif choice == "7":
    print("What are the top 20 nations that the players represent?")
    choice_7()

  elif choice == "8":
    print("What is the correlation between a player's potential and a player's age?")
    choice_8()
  
  elif choice == "9":
    print("What are the different positions and the number of players playing them?")
    choice_9()

  elif choice == "10":
    print("Correlation between age groups and wage")
    choice_10()
  
  elif choice == "11":
    print("Is there a significant pattern among the 15 most valuable players' age and wage?")
    choice_11()
  
  elif choice == "12":
    print("Correlation between weakfoot and passing, shooting, dribbling")
    choice_12()
  
  elif choice == "13":
    break

  else:
    print("Enter a valid number between 1 to 13")
    print("Please try again!")


print("Thank you for using our program!")
print("Group Name: Data Pirates")
print("Members: Matthew Krivitskiy, Jemil Patel, Manita Batas, Patricia Yrastorza")